In [ ]:
!pip install -U -q preprocessor
!pip install -U -q vaderSentiment
!pip install -U -q nltk
!pip install -U -q textblob
!pip install -U -q gensim
!pip install -U -q tqdm
!pip install -U -q bs4
!pip install python-Levenshtein
!pip install -U -q plotly

     |████████████████████████████████| 133kB 9.1MB/s 
     |████████████████████████████████| 1.5MB 9.0MB/s 
     |████████████████████████████████| 23.9MB 1.9MB/s 
     |████████████████████████████████| 81kB 4.8MB/s 
     |████████████████████████████████| 51kB 3.9MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149807 sha256=2d165ee66720c7626739b0c3bae39a7940fd0542926f26d87628b822e03fa4d5
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein
     |████████████████████████████████| 13.2MB 345kB/s 


In [ ]:
import preprocessor as p
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
import gensim
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import seaborn as sns
import plotly.io as pio
py.init_notebook_mode(connected=True)
# Required for plotly to run on colab
pio.renderers.default = 'colab'
%matplotlib inline

In [ ]:
tqdm.pandas()

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/MyDrive/CA683')

Mounted at /content/drive/


1. Dataframe with 16 M rows
2. Note - To process huge data of 3.9 GB with 16M rows of tweet data, this code was processed in Google Cloud machines with GPU
3. Drop useless columns

In [ ]:

df = pd.read_csv("tweets-bulk.csv", sep=';', nrows= 12143600, header=0, error_bad_lines=False) 
#df = pd.read_csv("tweet_1L.csv") -- 100,000 rows to test the processing with limited data set
data = df.drop(["url","fullname"],axis=1)


In [ ]:
data.head(2)
data.shape

(100000, 8)

Clean function using a combination of re and preprocessor

In [ ]:
def cleaning(text):
    cleantext = p.clean(text)
    cleantext = re.sub('[^a-zA-Z ]','',cleantext)
    return cleantext
def clean(data):
    #cleanr = re.compile("<.*?>")
    cleanr= re.sub(r"http\S+", "", data)
    
    cleanr= re.sub(r"pic.\S+", "", cleanr)
    #cleanr= re.sub(cleanr, "", data)
    #cleantext = re.sub(cleanr,'',data)
    cleantext = re.sub('[^a-zA-Z# ]','',cleanr)
    clean_hash = re.sub(r"#\S+","",cleantext)
    
    return clean_hash
data["text"] = data.progress_apply(lambda x:clean(str(x["text"])),axis=1)

data.head(2)

100%|██████████| 100000/100000 [00:02<00:00, 39636.17it/s]


,Unnamed: 0,id,user,timestamp,replies,likes,retweets,text
0,0,1132977055300300800,KamdemAbdiel,2019-05-27 11:49:14+00,0,0,0,appena uscito un nuovo video LES CRYPTOMONNAI...
1,1,1132977073402736640,bitcointe,2019-05-27 11:49:18+00,0,0,0,Cardano Digitize Currencies EOS ROI ATampT B...


tokenize and stemm and then join text

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
porter = PorterStemmer()

def stemSentence(sentence):
    #token words list
    tokens = [word.lower() for sent in nltk.sent_tokenize(sentence) for word in nltk.word_tokenize(sent)]
    stem_sentence=[]
    # Create list of word tokens after removing stopwords
    filtered_sentence =[] 
    #filtered_sentence = [w for w in token_words if not w in stopwords.words()]
    for word in tokens:
      if word not in stopwords:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


TextBlob functions

In [ ]:
data["subjectivity"] = np.zeros((data.shape[0],1))
data["blob_sent"] = np.zeros((data.shape[0],1))
def textblob_sent(text):
    text = stemSentence(text)
    analysis = TextBlob(str(text))
    return analysis.sentiment[0]
def textblob_sub(text):
    text = stemSentence(text)
    analysis = TextBlob(str(text))
    return analysis.sentiment[1]

In [ ]:
print(data["text"][6])
print(textblob_sent(data["text"][6]))
print(textblob_sent(stemSentence(data["text"][6])))

nwoodfine We have been building on the real  SVWhat have you been building on broken BTC
-0.1
-0.1


In [ ]:
data["blob_sent"] = data.progress_apply(lambda x:textblob_sent(str(x["text"])),axis=1)

100%|██████████| 100000/100000 [01:17<00:00, 1297.75it/s]


In [ ]:
data.to_csv("blob_sent_recorded_1L.csv")
data.head(10)

,Unnamed: 0,id,user,timestamp,replies,likes,retweets,text,subjectivity,blob_sent
0,0,1132977055300300800,KamdemAbdiel,2019-05-27 11:49:14+00,0,0,0,appena uscito un nuovo video LES CRYPTOMONNAI...,0.0,0.0
1,1,1132977073402736640,bitcointe,2019-05-27 11:49:18+00,0,0,0,Cardano Digitize Currencies EOS ROI ATampT B...,0.0,0.0
2,2,1132977023893139456,3eyedbran,2019-05-27 11:49:06+00,0,2,1,Another Test tweet that wasnt caught in the st...,0.0,0.0
3,3,1132977089089556481,DetroitCrypto,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices BTC USDETH USDLTC USD...,0.0,0.0
4,4,1132977092340191232,mmursaleen72,2019-05-27 11:49:23+00,0,0,0,Spiv Nosar Baz BITCOIN Is An Asset amp NOT A C...,0.0,0.0
5,5,1132977100309319680,0nurTOKA,2019-05-27 11:49:25+00,0,0,0,inceldii yerden kopsun bakalm ye kadar bir h...,0.0,0.0
6,6,1132977101638897665,evilrobotted,2019-05-27 11:49:25+00,0,0,0,nwoodfine We have been building on the real S...,0.0,-0.1
7,7,1132977108836323330,jabur_guilherme,2019-05-27 11:49:27+00,0,0,0,pedronauck como investidor vc um timo dev Sor...,0.0,0.0
8,8,1132977131850350593,INTBICON,2019-05-27 11:49:32+00,0,0,0,,0.0,0.0
9,9,1132977132714561536,MLWright15,2019-05-27 11:49:32+00,0,0,0,CHANGE IS COMINGGET READY Boom Another CB Jab ...,0.0,0.0


In [ ]:
data["subjectivity"] = data.progress_apply(lambda x:textblob_sub(x["text"]),axis=1)
data.to_csv("subjectivity_recorded_1L.csv")


100%|██████████| 100000/100000 [01:17<00:00, 1294.09it/s]


In [ ]:
data.shape

(100000, 10)

In [ ]:
index_names = data[ (data['subjectivity'] == 0.0) & (data['blob_sent'] == 0.000000)].index
  
# drop these given row indexes from dataFrame
data.drop(index_names, inplace = True)  

In [ ]:
data.shape

(39652, 10)

Vader sentiment

In [ ]:
sid = SentimentIntensityAnalyzer()
def vader(text):
    score = sid.polarity_scores(text)
    return score["compound"]


In [ ]:
data["vader_sent"] = data.progress_apply(lambda x:vader(str(x["text"])),axis=1)
data.head(5)

100%|██████████| 39652/39652 [00:07<00:00, 5178.28it/s]


,Unnamed: 0,id,user,timestamp,replies,likes,retweets,text,subjectivity,blob_sent,vader_sent
3,3,1132977089089556481,DetroitCrypto,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices BTC USDETH USDLTC USD...,0.400000,0.000000,0.0000
6,6,1132977101638897665,evilrobotted,2019-05-27 11:49:25+00,0,0,0,nwoodfine We have been building on the real S...,0.350000,-0.100000,-0.4767
14,14,1132922663113023489,CCNMarkets,2019-05-27 08:13:06+00,5,167,68,Bitcoin Price Hits in New High Whats Driving...,0.497273,0.148182,0.0000
17,17,1132977067048525824,malfouh,2019-05-27 11:49:17+00,1,1,1,The QLC airdrop tokens for every QLC holder...,0.850000,0.250000,0.0000
18,18,1132971551471603712,BoilingPointLTC,2019-05-27 11:27:22+00,1,19,6,You have roughly days left to get your tic...,0.200000,0.000000,0.0000


In [ ]:
adwords = ["give","referral","referal","bonus","signup","invite","freebitcoin","lucky",
          "lottery","giveaway","coupon","prizes","enter","airdrop","game","contest","bonuses"]
def contains_word(s, w):
    return (' ' + w + ' ') in (' ' + s + ' ')
def adchecker(text):
    words = []
    count = 0
    for i in adwords:
        if (contains_word(text,i)):
            count+= 1
            words.append(i)
    return count
def adword(text):
    words = []
    count = 0
    for i in adwords:
        if (contains_word(text,i)):
            count+= 1
            words.append(i)
    return words

In [ ]:
data["ad_count"] = np.zeros((data.shape[0],1))
data["adwords"] = np.zeros((data.shape[0],1))

data["ad_count"] = data.progress_apply(lambda x:adchecker(x["text"]),axis=1)
data["adwords"] = data.progress_apply(lambda x:adword(x["text"]),axis=1)

# data.to_csv("clean100.csv")


100%|██████████| 39652/39652 [00:00<00:00, 40204.41it/s]


In [ ]:
#classify if tweet is ad or not. Needs improvement
#0 is ad, 1 is normal
def ad_classifier(x):
    try:
        if x >1 :
            return 0
        else:
            return 1
    except:
        print(x)

data["ad_class"] = data.progress_apply(lambda x:ad_classifier(int(x["ad_count"])),axis=1)

100%|██████████| 39652/39652 [00:00<00:00, 69263.79it/s]


In [ ]:
data.astype({'ad_count': 'float64'}).dtypes
#nlarg = data.nlargest(30,"ad_count")
count = data["ad_class"].value_counts()
print(count)

1    39604
0       48
Name: ad_class, dtype: int64


In [ ]:
prominentusers = set(['iamjosephyoung', 'Bitcoin', 'BitcoinMagazine', 'btc', 'BTCNewsletter', 'BTCTN', 'coindesk', 'MoonOverlord',
                      'TuurDemeester', 'CryptoCoinsNews', 'LandM_Marius', 'kerooke', 'loomdart', 'GemCrypto', 
                      'coinnounce', 'MatiGreenspan', 'JWilliamsFstmed', 'VitalikButerin', 'VladZamfir', 'aantonop',
                      'ethereumJoseph', 'gavofyork', 'naval', 'tayvano_', 'NickSzabo4', 'cburniske',
                      'jwolpert', 'AndrewDARMACAP', 'ricburton', 'bcrypt', 'mikeraymcdonald',
                      'evan_van_ness', 'FEhrsam', 'JohnLilic', 'avsa', 'antiprosynth', 'nlw', 'brian_armstrong', 
                      'twobitidiot', 'APompliano', 'Cointelegraph', 'coinmetrics'])
def usernamer(x):
    if x in prominentusers:
        return 1
    else:
        return 0
data["prominent_user"] = data.progress_apply(lambda x:usernamer(x["user"]),axis=1)

data.head(5)


100%|██████████| 39652/39652 [00:00<00:00, 88625.10it/s]


,Unnamed: 0,id,user,timestamp,replies,likes,retweets,text,subjectivity,blob_sent,vader_sent,ad_count,adwords,ad_class,prominent_user
3,3,1132977089089556481,DetroitCrypto,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices BTC USDETH USDLTC USD...,0.400000,0.000000,0.0000,0,[],1,0
6,6,1132977101638897665,evilrobotted,2019-05-27 11:49:25+00,0,0,0,nwoodfine We have been building on the real S...,0.350000,-0.100000,-0.4767,0,[],1,0
14,14,1132922663113023489,CCNMarkets,2019-05-27 08:13:06+00,5,167,68,Bitcoin Price Hits in New High Whats Driving...,0.497273,0.148182,0.0000,0,[],1,0
17,17,1132977067048525824,malfouh,2019-05-27 11:49:17+00,1,1,1,The QLC airdrop tokens for every QLC holder...,0.850000,0.250000,0.0000,1,[airdrop],1,0
18,18,1132971551471603712,BoilingPointLTC,2019-05-27 11:27:22+00,1,19,6,You have roughly days left to get your tic...,0.200000,0.000000,0.0000,0,[],1,0


In [ ]:
keywords = ['onchain transaction', 'congestion', 'market cap','market cap drop', 'dispersion of returns', 'downturn', 'exchange volume', 'ATH', 'ath', 'atl', 'ATL',
            'bear trap', 'btfd', 'BTFD', 'buy wall', 'deflation', 'dump', 'fomo', 'FOMO', 'mooning', 'whale']
def keychecker(text):
    words = []
    count = 0
    for i in keywords:
        if (contains_word(str(text),i)):
            count+= 1
            words.append(i)
    return count
def keyword(text):
    words = []
    count = 0
    for i in keywords:
        if (contains_word(str(text),i)):
            count+= 1
            words.append(i)
    return words
def contains_word(s, w):
    return (' ' + w + ' ') in (' ' + s + ' ')

data["key_count"] = data.progress_apply(lambda x:keychecker(x["text"]),axis=1)
data["keywords"] = data.progress_apply(lambda x:keyword(x["text"]),axis=1)


100%|██████████| 39652/39652 [00:01<00:00, 33864.17it/s]


In [ ]:
print(data)
promu = data.nlargest(30,"key_count")
promu

       Unnamed: 0                   id  ... key_count keywords
3               3  1132977089089556481  ...         0       []
6               6  1132977101638897665  ...         0       []
14             14  1132922663113023489  ...         0       []
17             17  1132977067048525824  ...         0       []
18             18  1132971551471603712  ...         0       []
...           ...                  ...  ...       ...      ...
99989       99989  1127209437041713153  ...         0       []
99992       99992  1127209646358450177  ...         0       []
99993       99993  1127209718739607552  ...         0       []
99994       99994  1127209804521443329  ...         0       []
99998       99998  1127210238875193344  ...         0       []

[39652 rows x 17 columns]


,Unnamed: 0,id,user,timestamp,replies,likes,retweets,text,subjectivity,blob_sent,vader_sent,ad_count,adwords,ad_class,prominent_user,key_count,keywords
70962,70962,1127093880929832960,leb_crypto,2019-05-11 04:11:36+00,0,0,0,We may be seeing FOMO nowHowever Real FOMO kic...,0.300000,0.200000,0.0000,0,[],1,0,3,"[ATH, fomo, FOMO]"
9396,9396,1126844252347039744,JohnyCrypto1,2019-05-10 11:39:40+00,0,0,0,Breaking was very easy something doesnt seem ...,0.345238,0.078571,-0.3832,0,[],1,0,2,"[dump, FOMO]"
12892,12892,1126857412571324418,bukshukalaka,2019-05-10 15:05:43+00,0,1,1,jonathanaird fidelity of half their investors...,0.333333,-0.083333,0.0000,0,[],1,0,2,"[market cap, fomo]"
23065,23065,1126893173857554433,bitcoinization,2019-05-10 14:54:03+00,0,0,0,realKuzaG Agreed but I believe bitcoin will be...,0.677273,-0.181818,-0.5574,0,[],1,0,2,"[ATH, fomo]"
23826,23826,1126896492055289856,Norcalsteve22,2019-05-10 15:07:15+00,0,0,0,CNik coinbase You think the mil tether burn m...,0.666667,0.333333,-0.0972,0,[],1,0,2,"[dump, whale]"
33611,33611,1126928662874476551,bradmillscan,2019-05-10 17:15:05+00,0,0,0,whitebuffalo aantonop which is why I dont like...,0.330909,0.287273,0.4922,0,[],1,0,2,"[fomo, FOMO]"
37026,37026,1126943287191928838,BurniteGeorge,2019-05-10 18:13:11+00,0,0,0,Honestly dont know what to dump my open intoc...,0.410417,-0.085417,-0.2500,0,[],1,0,2,"[ATH, dump]"
45458,45458,1126983806114762753,rickeyjohnson12,2019-05-10 20:54:12+00,0,0,0,xtdisnkfe Yea that makes a lot of sense I thin...,0.557143,0.177143,0.7213,0,[],1,0,2,"[dump, FOMO]"
47837,47837,1126998115662192640,danielchiam7,2019-05-10 21:51:04+00,0,0,0,jebus Unless of course they already sold every...,0.100000,0.000000,-0.2960,0,[],1,0,2,"[ATH, FOMO]"
51487,51487,1127016181171310592,bukshukalaka,2019-05-10 23:02:51+00,0,0,0,To me of half fidelity trill investments is ...,0.166667,-0.166667,0.0000,0,[],1,0,2,"[market cap, FOMO]"


In [ ]:
def log_score(x):
    try:
        #x.astype(float)
        if int(x)<1:
            return 0
        else:
            return np.log(int(x))
    except:
        print(x)
        return 0
   # df[['Mean','Sum']] = df[['height_at_age_10','height_at_age_20']].apply(mean_and_sum(x),axis=1)
            #np.log(df.astype(float))
data["RT_score"] = data.progress_apply(lambda x:log_score(x["retweets"]),axis=1)
#data["Reply_score"] = data.progress_apply(lambda x:log_score(x["replies"]),axis=1)
data["fav_score"] = data.progress_apply(lambda x:log_score(x["likes"]),axis=1)
#data["RT_score"] = np.log(data["retweets"].astype(float))
data.head(2)

100%|██████████| 39652/39652 [00:00<00:00, 69929.87it/s]


,Unnamed: 0,id,user,timestamp,replies,likes,retweets,text,subjectivity,blob_sent,vader_sent,ad_count,adwords,ad_class,prominent_user,key_count,keywords,RT_score,fav_score
3,3,1132977089089556481,DetroitCrypto,2019-05-27 11:49:22+00,0,0,0,Current Crypto Prices BTC USDETH USDLTC USD...,0.40,0.0,0.0000,0,[],1,0,0,[],0.0,0.0
6,6,1132977101638897665,evilrobotted,2019-05-27 11:49:25+00,0,0,0,nwoodfine We have been building on the real S...,0.35,-0.1,-0.4767,0,[],1,0,0,[],0.0,0.0


In [ ]:
data["date"] = pd.to_datetime(data["timestamp"],errors= "coerce").dt.tz_localize(None)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39652 entries, 3 to 99998
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Unnamed: 0      39652 non-null  int64         
 1   id              39652 non-null  int64         
 2   user            39652 non-null  object        
 3   timestamp       39652 non-null  object        
 4   replies         39652 non-null  int64         
 5   likes           39652 non-null  int64         
 6   retweets        39652 non-null  int64         
 7   text            39652 non-null  object        
 8   subjectivity    39652 non-null  float64       
 9   blob_sent       39652 non-null  float64       
 10  vader_sent      39652 non-null  float64       
 11  ad_count        39652 non-null  int64         
 12  adwords         39652 non-null  object        
 13  ad_class        39652 non-null  int64         
 14  prominent_user  39652 non-null  int64         
 15  ke

In [ ]:
data['time'] = data['date']
data.index = data['time']


In [ ]:
# This is additional code to get twitter volume graph
sentiment_grouped_weekly = data.groupby(pd.Grouper(key="time", freq="1w")).agg(
    tweet_count_by_week = ('text', 'count')
)
drop_column_subse_week = sentiment_grouped_weekly.columns.values[1:]
sentiment_grouped_weekly.dropna(subset=drop_column_subse_week, inplace=True)

In [ ]:
sentiment_grouped_weekly.index
sentiment_grouped_weekly_data = sentiment_grouped_weekly.reset_index()
sentiment_grouped_weekly_data.time = pd.to_datetime(sentiment_grouped_weekly_data.time)
# sentiment_grouped_weekly_data.index = sentiment_grouped_weekly_data['time']

In [ ]:
sentiment_grouped_weekly_data.head()
# sentiment_grouped_weekly_data = sentiment_grouped_weekly_data[sentiment_grouped_weekly_data["tweet_count_by_week"] > 0]

,time,tweet_count_by_week
0,2012-08-05,1
1,2012-08-12,0
2,2012-08-19,0
3,2012-08-26,0
4,2012-09-02,0


In [ ]:
# plt.plot(sentiment_grouped_weekly_data, x='time')
# sentiment_grouped_weekly_data.plot(x='time')

btc_trace = go.Scatter(x=sentiment_grouped_weekly_data["time"], y=sentiment_grouped_weekly_data["tweet_count_by_week"], name= 'Tweet count')
py.iplot([btc_trace])

# plt.plot(sentiment_grouped_weekly_data["time"], sentiment_grouped_weekly_data["tweet_count_by_week"])
# plt.show()

In [ ]:
# This is additional code to get twitter volume graph

data.shape

data.head(5)
data["date2"] = pd.to_datetime(data["timestamp"],errors= "coerce").dt.date
data["month"] = pd.to_datetime(data["timestamp"],errors= "coerce").dt.month
data["hour"] = pd.to_datetime(data["timestamp"],errors= "coerce").dt.hour

date1 = data["date2"]
tweet_val = data["text"]

#data['date2'] = pd.to_datetime(data['date2'])

frame = { 'Date': date1, 'Tweet': tweet_val }
  
result = pd.DataFrame(frame)

#data_group = data_twitter_vol.groupby(data_twitter_vol['date2']).agg('count')
data_twitter_vol.head(5)

AttributeError: ignored

In [ ]:
def half(duration,x):
    try:
        if duration == "daily":
            #print(x)
            if int(x) >11:
                return 1
            else:
                return 0
        if duration =="hourly":
            if int(x) > 30:
                return 1
            else:
                return 0
    except:
        print(x)
        return 0
        
data["hour_score"] = data.progress_apply(lambda x:half("hourly",x["date"].minute),axis=1)
data["daily_score"] = data.progress_apply(lambda x:half("daily",x["date"].hour),axis=1)

100%|██████████| 3682809/3682809 [00:49<00:00, 73979.73it/s] 


In [ ]:
data["Daily Weight"] = (data["prominent_user"]+data["key_count"]+data["RT_score"]+data["fav_score"]+data["daily_score"])*data["ad_class"] #data["Reply_score"]
data["hourly Weight"] = (data["prominent_user"]+data["key_count"]+data["RT_score"]+data["fav_score"]+data["hour_score"])*data["ad_class"]  # #+data["Reply_score"]


In [ ]:
data["blob_sent"] = data["blob_sent"]*data["Daily Weight"]
data["vader_sent"] = data["vader_sent"]*data["hourly Weight"]

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3682809 entries, 3 to 12143593
Data columns (total 23 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              object        
 1   user            object        
 2   timestamp       object        
 3   replies         float64       
 4   likes           float64       
 5   retweets        float64       
 6   text            object        
 7   subjectivity    float64       
 8   blob_sent       float64       
 9   vader_sent      float64       
 10  ad_count        int64         
 11  adwords         object        
 12  ad_class        int64         
 13  prominent_user  int64         
 14  key_count       int64         
 15  keywords        object        
 16  RT_score        float64       
 17  fav_score       float64       
 18  date            datetime64[ns]
 19  hour_score      int64         
 20  daily_score     int64         
 21  Daily Weight    float64       
 22  hourly Weight   f

In [ ]:
# df = data.groupby(['date2','hour']).agg(['mean','count'])
# df.columns = [ ' '.join(str(i) for i in col) for col in df.columns]
# df.reset_index(inplace=True)
# print(df.columns)
# data['time'] = data['date']
# data.index = data['time']

sentiment_grouped_hourly = data.groupby(pd.Grouper(key="time", freq="1h")).agg(
    Daily_Weight_mean_by_hour = ('Daily Weight', np.mean),
    Daily_Weight_count_by_hour = ('Daily Weight', 'count'),
    blob_sent_mean_by_hour = ('blob_sent', np.mean),
    subjectivity_mean_by_hour = ('subjectivity', np.mean),
    vader_sent_mean_by_hour = ('vader_sent', np.mean)
)

In [ ]:
drop_column_subset = sentiment_grouped_hourly.columns.values[1:]
sentiment_grouped_hourly.dropna(subset=drop_column_subset, inplace=True)
sentiment_grouped_hourly.head()

,Daily_Weight_mean_by_hour,Daily_Weight_count_by_hour,blob_sent_mean_by_hour,subjectivity_mean_by_hour,vader_sent_mean_by_hour
time,,,,,
2009-02-18 05:00:00,7.896553,1,0.538401,0.477273,0.000000
2009-06-19 20:00:00,5.852030,1,4.291489,0.966667,2.699184
2010-03-11 05:00:00,2.302585,1,0.000000,0.500000,0.000000
2010-04-30 20:00:00,4.496508,1,1.911016,0.769444,1.545456
2010-06-19 23:00:00,5.584967,1,0.558497,0.400000,0.431159


In [ ]:
def movement_classifier(x):
    try:
        if x >= 0 :
            return 1
        else:
            return 0
    except:
        print(x)

In [ ]:
sentiment_grouped_hourly["tweet_diff"] = sentiment_grouped_hourly["Daily_Weight_count_by_hour"].diff()
sentiment_grouped_hourly["blob_sent_mean_by_hour_diff"] = sentiment_grouped_hourly["blob_sent_mean_by_hour"].diff()
sentiment_grouped_hourly["vader_sent_mean_by_hour_diff"] = sentiment_grouped_hourly["vader_sent_mean_by_hour"].diff()
sentiment_grouped_hourly["tweet_movement"] = sentiment_grouped_hourly["tweet_diff"].progress_apply(lambda x:movement_classifier(x))
sentiment_grouped_hourly["blob_sent_movement"] = sentiment_grouped_hourly["blob_sent_mean_by_hour_diff"].progress_apply(lambda x:movement_classifier(x))
sentiment_grouped_hourly["vader_sent_movement"] = sentiment_grouped_hourly["vader_sent_mean_by_hour_diff"].progress_apply(lambda x:movement_classifier(x))

100%|██████████| 55347/55347 [00:00<00:00, 777719.07it/s]


In [ ]:
sentiment_grouped_hourly.head()
sentiment_grouped_hourly.to_csv("twitter_sentiments_hourly_processed_data.csv")

In [ ]:
sentiment_grouped_hourly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 55347 entries, 2009-02-18 05:00:00 to 2019-07-25 00:00:00
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Daily_Weight_mean_by_hour     55347 non-null  float64
 1   Daily_Weight_count_by_hour    55347 non-null  int64  
 2   blob_sent_mean_by_hour        55347 non-null  float64
 3   subjectivity_mean_by_hour     55347 non-null  float64
 4   vader_sent_mean_by_hour       55347 non-null  float64
 5   tweet_diff                    55346 non-null  float64
 6   blob_sent_mean_by_hour_diff   55346 non-null  float64
 7   vader_sent_mean_by_hour_diff  55346 non-null  float64
 8   tweet_movement                55347 non-null  int64  
 9   blob_sent_movement            55347 non-null  int64  
 10  vader_sent_movement           55347 non-null  int64  
dtypes: float64(7), int64(4)
memory usage: 5.1 MB
